# Azimuthal Averaging

This guide showcases how to apply Azimuthal Averaging on 2D and 3D fields using UXarray.

## Azimuthal mean basics
An azimuthal average (or azimuthal mean) is a statistical measure that represents the average of a face-centered variable along rings/bands of constant distance from a specified central point. Azimuthal averaging is useful for describing circular/cylindrical features, where fields that strongly depend on distance from a center.

In UXarray, azimuthal averaging is non-conservative. This means that faces are assigned to radial bins (i.e., distance intervals from the central point) based only on the face center coordinate.

In [ ]:
# The imports below are used to visualize range rings in this notebook and
# are not needed for routine use of `azimuthal_mean()`.
import math
import operator
from functools import reduce

import cartopy.geodesic as gdyn
import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np

import uxarray as ux

In [ ]:
uxds = ux.open_dataset(
    "../../test/meshfiles/ugrid/outCSne30/outCSne30.ug",
    "../../test/meshfiles/ugrid/outCSne30/outCSne30_vortex.nc",
)

## 1. Azimuthally averaging an idealized 2D field

### Helper function to draw range rings and visualize azimuthal averaging:

In [ ]:
def hv_range_ring(lon, lat, rad_gcd, n_samples=2000, color="red", line_width=1):
    geo = gdyn.Geodesic()
    circ_pts = geo.circle(
        lon=lon, lat=lat, radius=rad_gcd * 111320, n_samples=n_samples
    )
    return hv.Path(circ_pts).opts(color=color, line_width=line_width)

### Step 1.1: Visualize the global field
The global field is shaded below using the UxDataArray.plot() accessor. To help visualize azimuthal averaging, the chosen central point is marked with an 'x' and rings are drawn at every 10 great-circle degrees from the central point (37°N, 1°E).

In [ ]:
# Display the global field
glob_plt = uxds["psi"].plot(
    cmap="inferno", periodic_elements="split", title="Global Field", dynamic=True
)

# Mark a center coordinate and draw range rings
lon, lat = 1, 37
glob_plt = glob_plt * hv.Points([(lon, lat)]).opts(
    color="lime", marker="x", size=10, line_width=2
)
glob_plt = glob_plt * reduce(
    operator.mul,
    [hv_range_ring(lon, lat, rr, color="lime") for rr in np.arange(10, 41, 10)],
)
glob_plt

### Step 1.2: Compute the azimuthal mean
Calling `.azimuthal_mean()` with the arguments below samples every 2° out to 40 great-circle degrees around the central point.

In [ ]:
azim_mean_psi, hits = uxds["psi"].azimuthal_mean(
    (lon, lat), 40.0, 2, return_hit_counts=True
)
azim_mean_psi

### Step 1.3: Plot the azimuthal mean
In the plot below, red dots mark where samples were taken at 2° intervals. The green vertical lines correspond to the green rings in the global field plot.

In [ ]:
plt.plot(azim_mean_psi["radius"], azim_mean_psi)
plt.scatter(azim_mean_psi["radius"], azim_mean_psi, s=10, color="red")
plt.xlabel("radius [great-circle degrees]")
plt.ylabel("psi")

[plt.axvline(rr, color="lime") for rr in np.arange(10, 41, 10)]

### Step 1.4: Inspect the hit count
The plot below shows the number of face centers that fall within each distance bin. As one would expect on a near-uniformly spaced mesh, the hit count increases linearly with distance.

In [ ]:
plt.plot(azim_mean_psi["radius"], hits)
plt.scatter(azim_mean_psi["radius"], hits, s=10, color="red")

## 2: Azimuthally averaging tropical cyclone fields
A high-resolution (~0.25°) aquaplanet general circulation model permits the development of a handful of strong tropical cyclones (TCs). Because TCs are fairly axisymmetric, azimuthal averaging is useful for transforming their fields into cylindrical coordinates and visualizing features such as their low central pressure and warm core.

In [ ]:
clon, clat = 114.54, -17.66
tcds = ux.open_dataset(
    "/glade/work/jpan/uxazim_demo/ne120np4_pentagons_100310.nc",
    "/glade/work/jpan/uxazim_demo/cam.h1i.plev.0013-01-13-00000.nc",
).squeeze()

### Step 2.1: Visualize the raw surface pressure field
Because we are only interested in a single tropical cyclone, we can subset a region of the global field using a bounding circle to speed up plotting.

In [ ]:
# Use a bounding circle to select faces whose centers lie
# within 5 great-circle degrees of the central point.
tcps = tcds["PS"].subset.bounding_circle((clon, clat), 5)

# Setting dynamic=True allows for quick, adaptive rendering as you zoom/pan.
# It also preserves the original mesh faces in the plot.
ps_plt = tcps.plot(
    cmap="inferno", periodic_elements="split", title="Surface pressure", dynamic=True
)
ps_plt

### Step 2.2: Compute the azimuthal mean of the 3D fields

In [ ]:
args = ((clon, clat), 3, 0.25)
azim_mean_T = tcds["T"].azimuthal_mean(*args)
azim_mean_Z = tcds["Z3"].azimuthal_mean(*args)
azim_mean_T

### Step 2.3: Plot the TC radial profile
The contour plot below shows the warm core and low pressure of the TC. After taking the azimuthal average, we subtract the value at the outermost radius to obtain an approximate anomaly relative to the ambient environment.

In [ ]:
# Subtract the value at the outer radius
Tpert = azim_mean_T - azim_mean_T.isel(radius=-1)
Zpert = azim_mean_Z - azim_mean_Z.isel(radius=-1)

plt.contour(
    azim_mean_Z["radius"],
    tcds["plev"] / 100,
    Zpert,
    levels=np.arange(-500, 501, 50),
    colors="black",
)
plt.contourf(
    azim_mean_T["radius"],
    tcds["plev"] / 100,
    Tpert,
    levels=np.arange(-10, 11, 2),
    cmap="bwr",
    extend="both",
)
plt.xlabel("distance from center [°]")
plt.ylim(1000, 100)
plt.yscale("log")
plt.ylabel("Pressure [hPa]")
plt.colorbar(label="T anomaly [K]")